In [1]:
import json
import os
import itertools
from typing import Tuple

import pandas as pd
import numpy as np
import talib as ta
from tqdm import tqdm

from ftx_client import FtxClient
import supertrend as spt
import backtesting as bt
from config import API_KEY, API_SECRET

In [2]:
with open("settings.json") as jsonfile:
    settings = json.load(jsonfile)
    
BACKTEST_FOLDER = settings["filepaths"]["backtest_folder"]
OPTIMIZEDML_FILEPATH = os.path.join(BACKTEST_FOLDER, settings["filepaths"]["optimized_ml_file"])
ANALYSIS_FILEPATH = os.path.join(BACKTEST_FOLDER, settings["filepaths"]["analysis_file"])

In [3]:
ftx = FtxClient(api_key=API_KEY, api_secret=API_SECRET)

In [4]:
markets = []
for future in ftx.list_futures():
    if future["type"] == "perpetual":
        if (future["volumeUsd24h"] > settings["markets"]["min_volume_usd_24h"] and 
                future["name"] not in settings["markets"]["blacklist"]):
            markets.append(future["name"])

In [5]:
optimzed_ml = pd.read_csv(OPTIMIZEDML_FILEPATH)

In [6]:
analysis_df = pd.DataFrame(columns=["Name", "AvgReturns", "StdDev", "AvgDrawdown", "RetDevRatio", "MinReturns", "MaxReturns",
             "AvgNegReturns", "AvgPosReturns", "PosNegRetRatio", "MedReturns", "MedNegReturns",
             "MedPosReturns", "MedPosNegRetRatio", "TheDfactor"])

pbar = tqdm(markets)
for market in pbar:
    pbar.set_description(market)
    df = ftx.get_historical_market_data(market, interval=settings["analysis"]["interval"], start_time=settings["analysis"]["start_time"])
    
    if len(df) < settings["analysis"]["min_data_length"]:
        continue
        
    multiplier = optimzed_ml.loc[optimzed_ml['Name'] == market]["Multiplier"].values[0]
    lookback = optimzed_ml.loc[optimzed_ml['Name'] == market]["Lookback"].values[0]
    
    profits = bt.backtest_dataframe(df, look_back=lookback, multiplier=multiplier)
    profits["Name"] = market
    analysis_df = analysis_df.append(profits, ignore_index=True)

XRP-PERP: 100%|██████████| 45/45 [02:37<00:00,  3.50s/it]  


In [7]:
analysis_df = analysis_df.round(1)
analysis_df.to_csv(ANALYSIS_FILEPATH, index=False)

In [8]:
pd.read_csv(ANALYSIS_FILEPATH).sort_values("TheDfactor", ascending=False) 

,Name,AvgReturns,StdDev,AvgDrawdown,RetDevRatio,MinReturns,MaxReturns,AvgNegReturns,AvgPosReturns,PosNegRetRatio,MedReturns,MedNegReturns,MedPosReturns,MedPosNegRetRatio,TheDfactor
27,MATIC-PERP,36.4,47.9,8.2,0.8,-0.7,119.5,-0.7,42.6,63.7,12.5,-0.7,15.2,22.7,2.8
7,BCH-PERP,26.0,34.2,3.1,0.8,-4.0,108.7,-2.9,34.3,12.0,12.9,-2.9,22.8,8.0,2.6
1,ADA-PERP,13.0,16.3,6.3,0.8,-5.7,45.4,-2.4,24.5,10.3,13.7,-1.4,19.5,14.2,2.2
25,LTC-PERP,15.1,30.9,4.5,0.5,-6.3,113.2,-3.0,30.6,10.2,0.4,-2.9,23.1,8.0,1.8
29,QTUM-PERP,30.6,39.8,6.6,0.8,-9.1,106.3,-5.5,57.7,10.6,16.0,-5.1,54.3,10.6,1.6
43,XRP-PERP,9.8,19.7,7.5,0.5,-7.8,58.1,-2.5,24.5,9.8,-0.6,-1.4,15.9,11.7,1.6
19,ETC-PERP,40.6,58.5,5.9,0.7,-4.5,175.3,-4.4,58.7,13.2,14.1,-4.4,40.3,9.1,1.5
24,LINK-PERP,12.6,25.0,5.2,0.5,-10.5,85.1,-4.6,32.6,7.1,-0.1,-3.8,26.9,7.1,1.4
2,ALPHA-PERP,58.1,119.3,7.7,0.5,-16.2,344.8,-7.8,107.5,13.8,0.7,-5.0,42.2,8.4,1.1
13,COMP-PERP,18.2,34.5,5.7,0.5,-14.5,107.3,-6.4,39.3,6.2,8.2,-4.1,25.9,6.3,1.1
